In [ ]:
import lymph

In [ ]:
graph_dict = {
    ('tumor', 'T')  : ['I', 'II', 'III', 'IV'], 
    ('lnl'  , 'I')  :       ['II'], 
    ('lnl'  , 'II') :       ['III'], 
    ('lnl'  , 'III'):       ['IV'], 
    ('lnl'  , 'IV') :       []
}

In [ ]:
model = lymph.models.Bilateral(graph_dict)
model

In [ ]:
model.set_params(spread=0.123)
model.contra.graph.edges["ItoII"].get_params("spread")

In [ ]:
model.set_params(ipsi_TtoIII_spread=0.234)
model.get_params(as_dict=True)

In [ ]:
model.set_modality("MRI", specificity=0.63, sensitivity=0.81)
model.set_modality("PET", specificity=0.86, sensitivity=0.79)
model.get_all_modalities()

In [ ]:
print(model.get_modality("PET").confusion_matrix)

In [ ]:
import pandas as pd

dataset_url = "https://raw.githubusercontent.com/rmnldwg/lydata/main/2021-usz-oropharynx/data.csv"
example_cols = [
    ("patient", "#", "age"),
    ("patient", "#", "hpv_status"),
    ("tumor", "1", "t_stage"),
    ("PET", "ipsi", "I"),
    ("PET", "ipsi", "II"),
    ("PET", "ipsi", "III"),
    ("PET", "ipsi", "IV"),
    ("MRI", "ipsi", "I"),
    ("MRI", "ipsi", "II"),
    ("MRI", "ipsi", "III"),
    ("MRI", "ipsi", "IV"),
]

usz_oropharynx = pd.read_csv(dataset_url, header=[0,1,2])
usz_oropharynx[example_cols]

In [ ]:
model.replace_all_modalities({})
model.set_modality("PET", specificity=0.86, sensitivity=0.79)

In [ ]:
model.load_patient_data(usz_oropharynx)
model.ipsi.patient_data["_model"]

In [ ]:
model.contra.patient_data["_model"]

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

max_time = model.max_time
time_steps = np.arange(max_time+1)
p = 0.4

early_prior = sp.stats.binom.pmf(time_steps, max_time, p)
model.set_distribution("early", early_prior)

In [ ]:
def late_binomial(support: np.ndarray, p: float = 0.5) -> np.ndarray:
    """Parametrized binomial distribution."""
    return sp.stats.binom.pmf(support, n=support[-1], p=p)

In [ ]:
model.set_distribution("late", late_binomial)
params_dict = model.get_params(as_dict=True, as_flat=True)
params_dict

In [ ]:
rng = np.random.default_rng(42)
test_probabilities = {p: rng.random() for p in params_dict}

llh = model.likelihood(given_param_kwargs=test_probabilities, log=True)
ipsi_llh = model.ipsi.likelihood(log=True)
contra_llh = model.contra.likelihood(log=True)

print(f"log-likelihood is {ipsi_llh:.2f} + {contra_llh:.2f} = {llh:.2f}")